In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import sys
import p5lib

%matplotlib inline

In [2]:
orig_data = pd.read_csv('MERGED2013_PP.csv')
print "Number of features: {}".format(len(orig_data.columns))
print "Number of rows: {}".format(len(orig_data))

Number of features: 1729
Number of rows: 7804


/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (7,9,1427,1542,1561,1575,1725,1726,1727,1728) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data = p5lib.preprocess_data(orig_data)
print "Num data after preprocessing: {}".format(len(data))
print "Num features: {}".format(len(data.columns))
print data.shape
print orig_data.shape

Num data after preprocessing: 6007
Num features: 42
(6007, 42)
(7804, 1729)


In [4]:
X = data[['CONTROL', 'DEBT_MDN', 'DEP_INC_AVG', 'GRAD_DEBT_MDN', 'IND_INC_AVG', 'INEXPFTE', 'PAR_ED_PCT_1STGEN', 
          'PAR_ED_PCT_HS', 'PAR_ED_PCT_MS', 'PAR_ED_PCT_PS', 'PCTFLOAN', 'PCTPELL', 'UG25abv', 'UGDS', 
          'WDRAW_DEBT_MDN', 'L4_COLLEGE', 'NPT4', 'NUM4', 
          'PFTFTUG1_EF', 'PFTFAC',
          'CCSIZSET', 
          'CCUGPROF', 
          'CCBASIC',
          'SAT_AVG_ALL', 'ACTCMMID', 'ADM_RATE_ALL', 'AVGFACSAL', 'COSTT4_A',
          'C150', 'RET_FT']].dropna()
y = X[['C150', 'RET_FT']]
X = X.drop('C150', 1)
X = X.drop('RET_FT', 1)
print X.shape
print y.shape

# print X.PFTFTUG1_EF.describe()

(1146, 28)
(1146, 2)


In [5]:
from sklearn import preprocessing

tmpX = X[['DEBT_MDN', 'DEP_INC_AVG', 'GRAD_DEBT_MDN', 'IND_INC_AVG', 'INEXPFTE', 'UGDS', 'WDRAW_DEBT_MDN', 'NPT4', 
          'NUM4', 'SAT_AVG_ALL', 'ACTCMMID', 'AVGFACSAL', 'COSTT4_A']]

scaler = preprocessing.MinMaxScaler()
scaledX = scaler.fit_transform(tmpX)

# print np.array(tmpX.ix[0])
# print scaledX[0]

# X_for_PCA = pd.DataFrame(scaledX)
pctX = X[['PAR_ED_PCT_1STGEN', 'PAR_ED_PCT_HS', 'PAR_ED_PCT_MS', 'PAR_ED_PCT_PS', 'PCTFLOAN', 'PCTPELL', 'UG25abv', 
          'PFTFTUG1_EF', 'PFTFAC', 'ADM_RATE_ALL']]
pctX = np.array(pctX)

forPcaX = np.concatenate((scaledX, pctX), axis=1)
# print scaledX.shape
# print pctX.shape
print forPcaX.shape


(1146, 23)


In [6]:
from sklearn.decomposition import PCA

pca = PCA(n_components=forPcaX.shape[1])
pca.fit(forPcaX)

# print pca.components_
print pca.explained_variance_ratio_


[  3.19854866e-01   1.75841822e-01   1.11443415e-01   8.04583938e-02
   7.50468911e-02   6.48022317e-02   3.82983398e-02   2.87615885e-02
   2.12016136e-02   1.73492140e-02   1.29063607e-02   1.03708261e-02
   9.88549630e-03   8.70803199e-03   7.40038246e-03   5.61296309e-03
   3.82064432e-03   3.24370756e-03   3.07195858e-03   1.26351122e-03
   6.57742160e-04   1.82785104e-19   9.81714648e-20]


In [7]:
pca = PCA(n_components=12)
pca.fit(forPcaX)
reducedX = pca.transform(forPcaX)
print reducedX.shape

(1146, 12)


In [8]:
# add the categorical feature

tmp1X = np.array(pd.get_dummies(X['CONTROL']))
tmp2X = X['L4_COLLEGE'].astype(int)
tmp2X = np.reshape(tmp2X, (len(tmp2X), 1)) 
tmp3X = np.array(pd.get_dummies(X['CCSIZSET']))
tmp4X = np.array(pd.get_dummies(X['CCUGPROF']))
tmp5X = np.array(pd.get_dummies(X['CCBASIC']))

print tmp1X.shape
print tmp2X.shape
print reducedX.shape
finalX = np.concatenate((reducedX, tmp1X, tmp2X, tmp3X, tmp4X), axis=1)
print finalX.shape

(1146, 3)
(1146, 1)
(1146, 12)
(1146, 45)


In [16]:
from sklearn import metrics
from sklearn import cross_validation as cv
from sklearn.tree import DecisionTreeRegressor

y = np.array(y)
X_train, X_test, y_train, y_test = cv.train_test_split(finalX, y, train_size=0.8)
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(916, 45)
(230, 45)
(916, 2)
(230, 2)


In [17]:
SVR_reg1, SVR_reg2 = p5lib.build_SVR_model(X_train, X_test, y_train, y_test)

--- Completion ---
{'epsilon': 0.10000000000000001, 'C': 0.10000000000000001, 'gamma': 0.10000000000000001}
R2 score on train data: 0.814329643919
R2 score on test  data: 0.790569146965
--- Retention ---
{'epsilon': 0.10000000000000001, 'C': 0.10000000000000001, 'gamma': 0.10000000000000001}
R2 score on train data: 0.674900144426
R2 score on test  data: 0.601001085102


In [11]:
DT_reg1, DT_reg2 = p5lib.build_DecisionTree_model(X_train, X_test, y_train, y_test)

--- Completion ---
{'max_depth': 4}
R2 score on train data: 0.795096939863
R2 score on test  data: 0.720806161046
--- Retention ---
{'max_depth': 4}
R2 score on train data: 0.657596303738
R2 score on test  data: 0.534489706059


In [14]:
KNN_reg1, KNN_reg2 = p5lib.build_KNN_model(X_train, X_test, y_train, y_test)

{'n_neighbors': 19}
R2 score on train data: 0.769164951082
R2 score on test  data: 0.745489246874
{'n_neighbors': 17}
R2 score on train data: 0.645862837857
R2 score on test  data: 0.640041457702


In [15]:
RForest_reg1, RForest_reg2 = p5lib.build_RandomForest_model(X_train, X_test, y_train, y_test)

R2 score on train data: 0.964493957909
R2 score on test  data: 0.784462021921
R2 score on train data: 0.942971826013
R2 score on test  data: 0.690069719749
